To be used for comparison purposes at a later time, so we can compare the difference between the flooded network
# Step 3: Cap Haitien GTFS Accessibility Analysis

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
from shapely.geometry import Point

In [3]:
import os, sys

In [4]:
# Get reference to GOSTNets
import sys
sys.path.append(r'C:\repos\GOSTnets')
import GOSTnets as gn

In [5]:
# read back your graph from step 1 from you saved pickle
G = nx.read_gpickle(r"C:\Users\war-machine\Documents\world_bank_work\haiti_gtfs_project\output_gtfs_graph_24hr_frame\cap_haitien_walk_w_ferries_via_osmnx.pickle")

In [6]:
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 12931
Number of edges: 34334
Average in degree:   2.6552
Average out degree:   2.6552


In [7]:
list(G.edges)[0:15]

[(247844908, 3806054858, 0),
 (330521785, 2353905402, 0),
 (330521785, 330540551, 0),
 (330521785, 330567999, 0),
 (330522696, 330569698, 0),
 (330522696, 3875208374, 0),
 (330522696, 3875208369, 0),
 (330523218, 3600503070, 0),
 (330523218, 2472620389, 0),
 (330523218, 2472620422, 0),
 (330523479, 330548415, 0),
 (330523479, 620028871, 0),
 (330523479, 2510265415, 0),
 (330523638, 2314667832, 0),
 (330523638, 2306542320, 0)]

## Identify only the largest graph

In [8]:
# compatible with NetworkX 2.4
list_of_subgraphs = list(G.subgraph(c).copy() for c in nx.weakly_connected_components(G))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i

# set your graph equal to the largest sub-graph
G_largest = max_graph

In [9]:
# print info about the largest sub-graph
print(nx.info(G_largest))

Name: 
Type: MultiDiGraph
Number of nodes: 12931
Number of edges: 34334
Average in degree:   2.6552
Average out degree:   2.6552


In [10]:
# inspect
# gn.save(G_largest,"gtfs_export_cap_haitian_merged_impute_walk_largest",r"C:\Users\war-machine\Documents\world_bank_work\haiti_gtfs_project\output_gtfs_graph_24hr_frame")

In [11]:
# insert origins
origins = gpd.read_file(r"C:\repos\GOST_PublicGoods\Implementations\gtfs_example\input_folder\cap_haitien_worldpop_pts2.shp")

In [12]:
# snap the origins to the road graph
snapped_origins = gn.pandana_snap(G, origins, source_crs = 'epsg:4326', target_crs = 'epsg:32619')

In [13]:
snapped_origins

,VALUE,geometry,NN,NN_dist
0,3.630365,POINT (-72.38833 19.81667),616755742,375.410681
1,3.881798,POINT (-72.38667 19.81667),616755742,351.747309
2,4.394933,POINT (-72.38250 19.81667),616755742,590.012777
3,4.479361,POINT (-72.38167 19.81667),616755742,662.445228
4,3.633273,POINT (-72.38833 19.81583),616755742,291.331109
...,...,...,...,...
182971,1.172181,POINT (-72.18667 19.49583),616038064,6766.523465
182972,1.175843,POINT (-72.18583 19.49583),616038064,6733.274830
182973,1.067793,POINT (-72.18500 19.49583),616038064,6701.005202
182974,1.201748,POINT (-72.18417 19.49583),616038064,6669.728790


In [14]:
# filter out origins with a distance more than 2km
snapped_origins_filtered = snapped_origins[snapped_origins.NN_dist < 2000]

In [15]:
snapped_origins_filtered

,VALUE,geometry,NN,NN_dist
0,3.630365,POINT (-72.38833 19.81667),616755742,375.410681
1,3.881798,POINT (-72.38667 19.81667),616755742,351.747309
2,4.394933,POINT (-72.38250 19.81667),616755742,590.012777
3,4.479361,POINT (-72.38167 19.81667),616755742,662.445228
4,3.633273,POINT (-72.38833 19.81583),616755742,291.331109
...,...,...,...,...
181500,8.905331,POINT (-71.74000 19.52250),2496045487,635.151029
181501,9.385510,POINT (-71.73917 19.52250),2496045487,583.872113
181593,8.597751,POINT (-71.74250 19.52167),2496045514,743.485636
181594,9.031757,POINT (-71.74167 19.52167),2496045514,753.712825


In [16]:
originNodes = list(snapped_origins_filtered['NN'].unique())

In [17]:
originNodes

[616755742,
 330739389,
 616556652,
 616556665,
 616553566,
 616556661,
 616553549,
 616556670,
 616553537,
 616556642,
 616556668,
 616722848,
 616722841,
 616722834,
 616722832,
 616722835,
 330748594,
 2322398723,
 2378164779,
 330728487,
 2378159627,
 7101639468,
 2378164775,
 2378159625,
 2378168605,
 2378168608,
 2378158461,
 763384845,
 2774734763,
 3158035816,
 2378159620,
 2225575669,
 330745690,
 2275968008,
 2225604782,
 620843684,
 330736645,
 330743585,
 2225575422,
 330747430,
 2225604715,
 620855485,
 620855507,
 330739265,
 620855481,
 620855480,
 7101671123,
 616085737,
 2239959280,
 6647538635,
 2322396300,
 7101671182,
 2239959285,
 2239959278,
 2239959275,
 620855475,
 2239959268,
 330751393,
 2285106105,
 2413706138,
 620855473,
 620855476,
 2413706133,
 620855897,
 620855464,
 620855460,
 2322391135,
 330743292,
 2385795482,
 2275907685,
 620843229,
 7101671051,
 2240032201,
 2413706127,
 6733460167,
 620843224,
 620843234,
 617270697,
 3448749786,
 620855452,
 65

In [18]:
# insert origins
destinations = gpd.read_file(r"C:\repos\GOST_PublicGoods\Implementations\gtfs_example\input_folder\cap_haitien_health_pts.shp")

In [19]:
# snap the destinations to the road graph
snapped_destinations = gn.pandana_snap(G, destinations, source_crs = 'epsg:4326', target_crs = 'epsg:32619')

In [20]:
snapped_destinations

,Code2018,Facility n,Department,Vilcom,Commune,Facility t,MGA,Geolocatio,Geolocat_1,geometry,NN,NN_dist
0,476,HôPital Universitaire Justinien,Nord,11,Cap-Haitien,hopital universitaire,publique,19.762251,-72.206200,POINT (-72.20620 19.76225),614883096,40.575748
1,573,Centre Sante Communautaire De Ferrier (Csc),Nord'Est,12,Ferrier,dispensaire/centre communautaire de sante,publique,19.616028,-71.779961,POINT (-71.77996 19.61603),617490075,37.324084
2,593,Centre De Santé De Sainte Suzanne,Nord'Est,32,Sainte-Suzanne,centre de sante sans lit,publique,19.584867,-72.089272,POINT (-72.08927 19.58487),2295055691,121.894915
3,596,Centre Sante Communautaire De Phaeton,Nord'Est,33,Terrier-Rouge,dispensaire/centre communautaire de sante,publique,19.676502,-71.895271,POINT (-71.89527 19.67650),2314667846,58.595632
4,578,Centre Medico Social De Ouanaminthe,Nord'Est,21,Ouanaminthe,centre de sante avec lit,publique,19.549278,-71.720268,POINT (-71.72027 19.54928),2482947395,35.096853
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2157,Clinique Sante De Siloé,Nord,11,Cap-Haitien,centre de sante sans lit,mixte,19.732412,-72.227005,POINT (-72.22701 19.73241),619371909,59.788657
96,2149,New Hope Hospital,Nord,22,Plaine-du-Nord,hopital communautaire de reference,mixte,19.681231,-72.279991,POINT (-72.27999 19.68123),2293953844,746.018265
97,560,Hopital Saint Jean De Limbé,Nord,61,Limbe,hopital communautaire de reference,mixte,19.707607,-72.401764,POINT (-72.40176 19.70761),618265398,34.823633
98,2133,Ofatma Cap Haitien,Nord,11,Cap-Haitien,hopital communautaire de reference,mixte,19.763382,-72.205795,POINT (-72.20580 19.76338),647091283,45.149034


In [21]:
destinationsNodes = list(snapped_destinations['NN'].unique())

## Calculate OD Matrix

In [22]:
OD_matrix = gn.calculate_OD(G, originNodes, destinationsNodes, fail_value=-1, weight='length')

In [23]:
OD_matrix

array([[38628.341, 87300.849, 62123.44 , ..., 16741.623, 38651.988,
        69439.106],
       [37719.1  , 86391.608, 61214.199, ..., 15832.382, 37742.747,
        68529.865],
       [38282.693, 86955.201, 61777.792, ..., 16395.975, 38306.34 ,
        69093.458],
       ...,
       [66461.709, 15714.392, 51925.435, ..., 87081.941, 66485.356,
        49317.347],
       [66876.814, 16129.497, 52340.54 , ..., 87497.046, 66900.461,
        49732.452],
       [66508.487, 15761.17 , 51972.213, ..., 87128.719, 66532.134,
        49364.125]])

In [27]:
# metrics
type(OD_matrix)
avg_trip_time = np.mean(OD_matrix)
avg_trip_time

33061.10119967321

In [28]:
print(f"average trip time is: {avg_trip_time/60} minutes")

average trip time is: 551.0183533278869 minutes


## calculate accessibility

In [225]:
# For each row, the closest facility is the smallest value in the row
closest_facility_per_origin = OD_matrix.min(axis=1)

results = pd.DataFrame([originNodes, closest_facility_per_origin]).transpose()

colName = "travel_time_to_closest_facility"
results.columns = ['NN', colName]

In [226]:
closest_facility_per_origin

array([ 727.3928,    0.    ,  450.8744, ..., 2690.384 , 3022.468 ,
       2727.8064])

In [227]:
results

,NN,travel_time_to_closest_facility
0,6.16756e+08,727.393
1,3.30739e+08,0
2,6.16557e+08,450.874
3,6.16557e+08,396.162
4,6.16554e+08,208.108
...,...,...
8882,2.49605e+09,2688.89
8883,2.49635e+09,2745.27
8884,2.49605e+09,2690.38
8885,2.49635e+09,3022.47


In [228]:
output = snapped_origins_filtered.copy()
output = pd.merge(output, results, on="NN")

In [229]:
output

,VALUE,geometry,NN,NN_dist,travel_time_to_closest_facility
0,3.630365,POINT (-72.38833 19.81667),616755742,375.410681,727.393
1,3.881798,POINT (-72.38667 19.81667),616755742,351.747309,727.393
2,4.394933,POINT (-72.38250 19.81667),616755742,590.012777,727.393
3,4.479361,POINT (-72.38167 19.81667),616755742,662.445228,727.393
4,3.633273,POINT (-72.38833 19.81583),616755742,291.331109,727.393
...,...,...,...,...,...
162507,8.280589,POINT (-71.75333 19.52500),2496352697,209.087792,3022.47
162508,7.810264,POINT (-71.75250 19.52417),2496352697,138.256164,3022.47
162509,8.254992,POINT (-71.75250 19.52500),2496352745,149.820614,2727.81
162510,8.222400,POINT (-71.75167 19.52500),2496352745,63.375339,2727.81


In [230]:
#convert travel_time_to_closest_facility to number
output["travel_time_to_closest_facility"] = pd.to_numeric(output["travel_time_to_closest_facility"])

In [231]:
# save a shapefile...
destinations_gpd = gpd.GeoDataFrame(output, crs = "epsg:4326", geometry = 'geometry')
#destinations_gpd.to_file(r"C:\repos\GOST_PublicGoods\Implementations\gtfs_example\output_folder\cap_haitien_accessibility_health_walk_only.shp")

## Testing: find a shortest path

In [232]:
nx.dijkstra_path(G, 763384845, 2240032201, weight='length')

[763384845, 2225575422, 616085737, 6970559909, 2240032154, 2240032201]

In [233]:
# Look up route cost via graph
nx.dijkstra_path_length(G, 763384845, 2240032201, weight='length')

969.01952

In [234]:
nx.dijkstra_path(G, 629215322, 2333192306, weight='length')

[629215322,
 'XHQ66_stop_0150',
 'XHQ66_stop_0104',
 'XHQ66_stop_0121',
 2333192291,
 2333192306]

In [235]:
nx.dijkstra_path(G, 2333192306,629215322, weight='length')

[2333192306,
 2333192291,
 'XHQ66_stop_0200',
 'XHQ66_stop_0156',
 'XHQ66_stop_0123',
 'XHQ66_stop_0004',
 'XHQ66_stop_0041',
 629215322]